## Import the libraries

In [26]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend
import tensorflow_datasets as tfds
import cv2
from PIL import Image
import os 
import time

## Load and process the data

In [4]:
# Reading the input images and putting them into a numpy array
data=[]
labels=[]

height = 150
width = 150
channels = 3
classes = 2
n_inputs = height * width * channels

for i in range(0, classes, 1):
    path = "Data/{}/".format(i)
    print("Loading Class {}...".format(i)+path)
    Class=os.listdir(path)
    for a in Class:
        try:
            image=cv2.imread(path+a)
            size_image = cv2.resize(image, (height, width))
            data.append(np.array(size_image))
            labels.append(i)
        except AttributeError:
            print("Attribut Error")
            
Cells=np.array(data)
labels=np.array(labels)

Loading Class 0...Data/0/
Loading Class 1...Data/1/


In [5]:
#Randomize the order of the input images
s=np.arange(Cells.shape[0])
np.random.seed(classes)
np.random.shuffle(s)
Cells=Cells[s]
labels=labels[s]

In [6]:
#Spliting the images into train and validation sets
(X_train,X_val)=Cells[(int)(0.2*len(labels)):],Cells[:(int)(0.2*len(labels))]
X_train = X_train.astype('float32')/255 
X_val = X_val.astype('float32')/255
(y_train,y_val)=labels[(int)(0.2*len(labels)):],labels[:(int)(0.2*len(labels))]


In [7]:
#Using one hot encoding for the train and validation labels
y_train = tf.keras.utils.to_categorical(y_train, classes)
y_val = tf.keras.utils.to_categorical(y_val, classes)

print(X_train.shape)

(349, 150, 150, 3)


## Build and compile the model

In [8]:
#Build the model
model = tf.keras.models.Sequential()
model.add(keras.layers.Conv2D(filters=16, kernel_size=(5,5), activation='relu', input_shape=X_train.shape[1:]))
model.add(keras.layers.MaxPool2D(pool_size=(2, 2)))
model.add(keras.layers.Dropout(rate=0.25))
model.add(keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'))
model.add(keras.layers.MaxPool2D(pool_size=(2, 2)))
model.add(keras.layers.Dropout(rate=0.25))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(256, activation='relu'))
model.add(keras.layers.Dropout(rate=0.5))
model.add(keras.layers.Dense(classes, activation='softmax'))
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 146, 146, 16)      1216      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 73, 73, 16)        0         
_________________________________________________________________
dropout (Dropout)            (None, 73, 73, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 71, 71, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 35, 35, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 35, 35, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 39200)             0

In [9]:
#Compilation of the model
model.compile(
    loss='categorical_crossentropy', 
    optimizer='adam', 
    metrics=['accuracy']
)

## Train the model

In [10]:
#using ten epochs for the training and saving the accuracy for each epoch
epochs = 10
history = model.fit(X_train, y_train, batch_size=32, epochs=epochs,
validation_data=(X_val, y_val))


Epoch 1/10
11/11 [==============================] - 14s 1s/step - loss: 4.4567 - accuracy: 0.5673 - val_loss: 0.7943 - val_accuracy: 0.6092
Epoch 2/10
11/11 [==============================] - 0s 43ms/step - loss: 0.6441 - accuracy: 0.7163 - val_loss: 0.3812 - val_accuracy: 0.9770
Epoch 3/10
11/11 [==============================] - 0s 42ms/step - loss: 0.2610 - accuracy: 0.9198 - val_loss: 0.2422 - val_accuracy: 0.9540
Epoch 4/10
11/11 [==============================] - 0s 45ms/step - loss: 0.1319 - accuracy: 0.9628 - val_loss: 0.0939 - val_accuracy: 0.9770
Epoch 5/10
11/11 [==============================] - 0s 43ms/step - loss: 0.0665 - accuracy: 0.9685 - val_loss: 0.0518 - val_accuracy: 0.9885
Epoch 6/10
11/11 [==============================] - 0s 43ms/step - loss: 0.0568 - accuracy: 0.9828 - val_loss: 0.0452 - val_accuracy: 1.0000
Epoch 7/10
11/11 [==============================] - 0s 43ms/step - loss: 0.0247 - accuracy: 0.9914 - val_loss: 0.0493 - val_accuracy: 0.9655
Epoch 8/10
11/

## Evaluate the model on test data

In [54]:
#Load test data
y_test=pd.read_csv("./TrafficSigns_dataset/Test.csv")
labels=y_test['Path'].values
y_test=y_test['ClassId'].values

print (y_test)

data=[]

for f in labels:
    image=cv2.imread('./TrafficSigns_dataset/test/'+f.replace('Test/', ''))
    image_from_array = Image.fromarray(image, 'RGB')
    size_image = image_from_array.resize((height, width))
    data.append(np.array(size_image))

X_test=np.array(data)
X_test = X_test.astype('float32')/255 

[16  1 38 ...  6  7 10]


In [55]:
#Predict output for test examples
pred = model.predict_classes(X_test)

In [56]:
#Calculate accuracy
correct = 0
incorrect = 0
if(len(pred) == len(y_test)):
    for i in range(0, len(pred)):
        if (pred[i] == y_test[i]):
            correct+=1
        else:
            incorrect+=1

test_acc = correct/(correct+incorrect)
print('Test accuracy:', test_acc)

Test accuracy: 0.9355502771179731


## Save the model

In [19]:
model.save("WhiteBox_CNN.h5")

In [36]:
import utils
output_filename = "output.jpeg"
image_filename = "Test.jpg"
class_label = 0
grad_CAM_map= utils.grad_CAM_plus(image_filename, class_label, output_filename, model)
grad_CAM_plus(filename, label_id, output_filename, model)

AttributeError: module 'tensorflow' has no attribute 'get_default_graph'

In [37]:
def load_image(path):
    img = skimage.io.imread(path)
    img = img / 255.0
    assert (0 <= img).all() and (img <= 1.0).all()
    short_edge = min(img.shape[:2])
    yy = int((img.shape[0] - short_edge) / 2)
    xx = int((img.shape[1] - short_edge) / 2)
    crop_img = img[yy: yy + short_edge, xx: xx + short_edge]
    resized_img = skimage.transform.resize(crop_img, (150, 150))
    return resized_img

In [ ]:
def grad_CAM_plus(filename, label_id, output_filename, model):
    
    img1 = load_image(filename)
    model(img1)
    cost = tf.nn.softmax(model.get_layer(name="dense_1"))

    target_conv_layer = model.get_layer(name="conv2d_1").output
    target_conv_layer_grad = tf.gradients(cost, target_conv_layer)[0]

    first_derivative = tf.exp(cost)[0][label_id]*target_conv_layer_grad

    second_derivative = tf.exp(cost)[0][label_id]*target_conv_layer_grad*target_conv_layer_grad 

    triple_derivative = tf.exp(cost)[0][label_id]*target_conv_layer_grad*target_conv_layer_grad*target_conv_layer_grad  

    output = [0,0]
    output[label_id] = 1.0
    output = np.array(output)
    print (label_id)
    conv_output, conv_first_grad, conv_second_grad, conv_third_grad = sess.run([target_conv_layer, first_derivative, second_derivative, triple_derivative], feed_dict={input_image:[img1], label_index:label_id, label_vector: output.reshape((1,-1))})

    global_sum = np.sum(conv_output[0].reshape((-1,conv_first_grad[0].shape[2])), axis=0)

    alpha_num = conv_second_grad[0]
    alpha_denom = conv_second_grad[0]*2.0 + conv_third_grad[0]*global_sum.reshape((1,1,conv_first_grad[0].shape[2]))
    alpha_denom = np.where(alpha_denom != 0.0, alpha_denom, np.ones(alpha_denom.shape))
    alphas = alpha_num/alpha_denom

    weights = np.maximum(conv_first_grad[0], 0.0)
    alphas_thresholding = np.where(weights, alphas, 0.0)
    alpha_normalization_constant = np.sum(np.sum(alphas_thresholding, axis=0),axis=0)
    alpha_normalization_constant_processed = np.where(alpha_normalization_constant != 0.0, alpha_normalization_constant, np.ones(alpha_normalization_constant.shape))
    alphas /= alpha_normalization_constant_processed.reshape((1,1,conv_first_grad[0].shape[2]))
    deep_linearization_weights = np.sum((weights*alphas).reshape((-1,conv_first_grad[0].shape[2])),axis=0)
    grad_CAM_map = np.sum(deep_linearization_weights*conv_output[0], axis=2)
    cam = np.maximum(grad_CAM_map, 0)
    cam = cam / np.max(cam) # scale 0 to 1.0   
    print("\nCam Shape Before:")
    print(cam.shape)
    cam = resize(cam, (150,150))
    gb = guided_BP([img1], label_id)
    visualize(img1, cam, output_filename, gb) 
    return cam